# Data collection
Gather public SPP Weis data from https://marketplace.spp.org/groups/operational-data-weis

In [ ]:
import os
import pandas as pd
import duckdb

import requests
from io import StringIO

import ibis
import ibis.selectors as s
ibis.options.interactive = True

# logging
import logging

In [ ]:
# define log
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)


In [ ]:
os.chdir('../..')

In [ ]:
import src.data_collection as dc

## Mid Term Load Forecast

![_](../../imgs/mtlf.PNG)

HOUR = {0000, ..., 2300}
DAY = {01, ..., 31}

In [ ]:
# weis time are in central time
pd.to_datetime("4/1/2023 07:00:00").tz_localize("America/Chicago")

In [ ]:
dc.get_time_components('4/1/2023 07:30')

In [ ]:
dc.get_time_components('4/1/2023 07:30', five_min_ceil=True)

In [ ]:
tc = dc.get_time_components(five_min_ceil=True)
tc

In [ ]:
tc = dc.get_time_components(five_min_ceil=False)
tc

In [ ]:
tc = dc.get_time_components('6/7/2023 08:00:00')

In [ ]:
mtlf_url = dc.get_hourly_mtlf_url(tc)
mtlf_url

In [ ]:
# test error handling
df = dc.get_csv_from_url(mtlf_url+'bad_url')
df

In [ ]:
# test error handling
df = dc.get_csv_from_url('a'+mtlf_url)
df

In [ ]:
# test success
df = dc.get_csv_from_url(mtlf_url)
df

In [ ]:
# test getting and processing
dc.get_process_mtlf(tc)

In [ ]:
# test getting data from a range of timestamps
end_ts = (
    pd.Timestamp.utcnow().tz_convert("America/Chicago").tz_localize(None) - 
    pd.Timedelta('2D')
).tz_localize(None)

range_df = dc.get_range_data_mtlf(end_ts=end_ts,  n_periods=3)

range_df

In [ ]:
range_df.info()

In [ ]:
range_df.to_parquet('data/mtlf.parquet')

In [ ]:
end_ts = pd.Timestamp.utcnow().tz_convert("America/Chicago").tz_localize(None)
new_range_df = dc.get_range_data_mtlf(end_ts=end_ts,  n_periods=3)
new_range_df.to_parquet('data/mtlf_temp.parquet')

In [ ]:
con = ibis.duckdb.connect("data/spp.ddb")
if 'mtlf' in con.list_tables():
    con.drop_table('mtlf')

In [ ]:
dc.upsert_mtlf(range_df)

In [ ]:
dc.upsert_mtlf(new_range_df)

In [ ]:
dc.upsert_mtlf(new_range_df)

In [ ]:
# reconnect to drop temp tables
con = ibis.duckdb.connect("data/spp.ddb")
con.list_tables()

In [ ]:
# test collecting and upserting
# this function does it all and can easily 
# be used in an automated job
dc.collect_upsert_mtlf(n_periods=20)

In [ ]:
con.table('mtlf').to_pandas().sort_values('GMTIntervalEnd')

## Mid Term Resource Forecast

![_](../../imgs/mtrf.PNG)

In [ ]:
tc = dc.get_time_components('6/7/2023 08:00:00')
tc

In [ ]:
mtrf_url = dc.get_hourly_mtrf_url(tc)
mtrf_url

In [ ]:
df = dc.get_csv_from_url(mtrf_url)
df

In [ ]:
# single file
tc = dc.get_time_components('6/7/2023 08:00:00')
dc.get_process_mtrf(tc)

In [ ]:
# test getting data from a range of timestamps
end_ts = pd.Timestamp.utcnow().tz_convert("America/Chicago").tz_localize(None)
range_df = dc.get_range_data_mtrf(end_ts=end_ts,  n_periods=3)

range_df

In [ ]:
# test collecting and upserting
# this function does it all and can easily 
# be used in an automated job
dc.collect_upsert_mtrf(n_periods=20)

## LMP settlement location prices

![_](../../imgs/lmp_settlement_location.PNG)

In [ ]:
time_str = str(pd.Timestamp.now())
time_str

In [ ]:
dc.get_time_components(time_str)

In [ ]:
time_str_list = [str(dt) for dt in pd.date_range(end=pd.Timestamp.now(), periods=8, freq='H')]
time_str_list

In [ ]:
tc = dc.get_time_components('6/8/2023 08:21:00', five_min_ceil=True)
tc

In [ ]:
lmp_url = dc.get_5min_lmp_url(tc)
lmp_url

In [ ]:
lmp = dc.get_csv_from_url(lmp_url)
lmp

In [ ]:
# single file
# tc = dc.get_time_components('6/7/2023 08:00:00')
# dc.get_process_5min_lmp(tc)

In [ ]:
tc = dc.get_time_components('6/8/2023 08:21:00', five_min_ceil=True)
tc

In [ ]:
df_lmp = dc.get_process_5min_lmp(tc)
df_lmp

In [ ]:
# multiple interval files
end_ts = pd.Timestamp.now()# + pd.Timedelta('1H')
end_ts

In [ ]:
df_lmp = dc.get_range_data_interval_5min_lmps(end_ts, n_periods=8)

In [ ]:
df_lmp

In [ ]:
group_cols = [
        'Interval_HE', 'GMTIntervalEnd_HE', 'timestamp_mst_HE',
        'Settlement_Location_Name', 'PNODE_Name'
    ]
value_cols = ['LMP', 'MLC', 'MCC', 'MEC']

(
    df_lmp[group_cols + value_cols]
    .groupby(group_cols)
    .mean()
    .reset_index()
)

### Test LMP daily file

In [ ]:
tc = dc.get_time_components('6/1/2024 23:00:00', five_min_ceil=True)
tc

In [ ]:
df_lmp = dc.get_process_daily_lmp(tc)

In [ ]:
df_lmp

In [ ]:
df_lmp.info()

In [ ]:
# multiple daily files
end_ts = pd.Timestamp('6/2/2024 00:00:00')# + pd.Timedelta('1H')
end_ts

In [ ]:
df_lmp = dc.get_range_data_interval_daily_lmps(end_ts, n_periods=7)
df_lmp

In [ ]:
# con.drop_table('test_lmp')

In [ ]:
# con.create_table('test_lmp', df_lmp)

In [ ]:
end_ts = pd.Timestamp('6/1/2024 23:00:00')# + pd.Timedelta('1H')
end_ts

In [ ]:
# test interval file
dc.collect_upsert_lmp(daily_file=False, end_ts=end_ts)

In [ ]:
# test interval file
dc.collect_upsert_lmp(daily_file=True, end_ts=end_ts)

In [ ]:
con.list_tables()

In [ ]:
con.table('lmp')

In [ ]:
# con.drop_table('mtrf')

## Generation capacity by fuel type

In [ ]:
tc = dc.get_time_components('11/8/2024 23:00:00')
tc

In [ ]:
get_gen_cap_url(tc)

In [ ]:
def get_process_gen_cap(tc: dict) -> pd.DataFrame:
    """
    Function to get and process MTLF data.
    Args:
        tc: dict - dictionary returned from get_time_components()
    Returns:
        pd.DataFrame with processed data for file corresponding to
            url created from tc
    """
    gen_cap_url = get_gen_cap_url(tc)
    log.debug(f'gen_cap_url: {gen_cap_url}')

    df = dc.get_csv_from_url(gen_cap_url)

    if df.shape[0] > 0:
        dc.format_df_colnames(df)
        df['GMT_TIME'] = pd.to_datetime(df['GMT_TIME'], format='ISO8601')
        df.rename(columns={'GMT_TIME': 'GMTIntervalEnd'}, inplace=True)
        df['timestamp_mst'] = (
            df.GMTIntervalEnd
            .dt.tz_convert('MST')
            .dt.tz_localize(None)
        )

    return df

In [ ]:
df_gen_cap = dc.get_process_gen_cap(tc)#.info()

In [ ]:
df_gen_cap.info()

In [ ]:
# multiple daily files
end_ts = pd.Timestamp('6/2/2024 00:00:00')# + pd.Timedelta('1H')
end_ts

In [ ]:
df_gen_cap = dc.get_range_data_gen_cap(end_ts=end_ts,  n_periods=3)

In [ ]:
df_gen_cap.head(2)

In [ ]:
def upsert_gen_cap(
        gen_cap_upsert: pd.DataFrame,
        backfill: bool=False,
) -> None:
    """
    Function to upsert new/backfilled generation capacity data into duckdb database.
    Args:
        gen_cap_upsert: pd.DataFrame - dataframe to upsert to MTRF table in database.
        backfill: bool = False - if true removes rows with missing values before
            upsert.  This removes rows where average actual is missing because
            the time period is forecasted and prevents overwriting actual values
            with forecasted values.
    Returns:
        None - new data is upserted to table
    """
    # remove missing values if backfilling
    if backfill:
        gen_cap_upsert.dropna(axis=0, how='any', inplace=True)
    # remove any duplicated primary keys
    gen_cap_upsert = gen_cap_upsert[~gen_cap_upsert.GMTIntervalEnd_HE.duplicated()]
    update_count = len(gen_cap_upsert)
    # NOTE: the df col order must match the order in the table
    ordered_cols = [
        'GMTIntervalEnd_HE', 'timestamp_mst',
        'Coal_Market', 'Coal_Self', 'Hydro', 
        'Natural_Gas', 'Nuclear', 'Solar', 'Wind',
    ]
    gen_cap_upsert = gen_cap_upsert[ordered_cols]
    log.info(f'gen_cap_upsert.timestamp_mst_HE.min(): {gen_cap_upsert.timestamp_mst_HE.min()}')
    log.info(f'gen_cap_upsert.timestamp_mst_HE.max(): {gen_cap_upsert.timestamp_mst_HE.max()}')

    # upsert with duckdb
    with duckdb.connect('~/spp_weis_price_forecast/data/spp.ddb') as con_ddb:
        create_gen_cap = '''
        CREATE TABLE IF NOT EXISTS gen_cap (
             GMTIntervalEnd_HE TIMESTAMP PRIMARY KEY,
             timestamp_mst_HE TIMESTAMP,
             Coal_Market DOUBLE, 
             Coal_Self DOUBLE,
             Hydro DOUBLE,
             Natural_Gas DOUBLE,
             Nuclear DOUBLE,
             Solar DOUBLE,
             Wind DOUBLE
             );
        '''
        con_ddb.sql(create_gen_cap)

        res = con_ddb.sql('select count(*) from gen_cap')
        start_count = res.fetchall()[0][0]
        log.info(f'starting count: {start_count:,}')

        gen_cap_insert_update = '''
        INSERT INTO gen_cap
            SELECT * FROM gen_cap_upsert
            ON CONFLICT DO UPDATE SET Coal_Market = EXCLUDED.Coal_Market, 
            Coal_Self = EXCLUDED.Coal_Self,
            Hydro = EXCLUDED.Hydro,
            Natural_Gas = EXCLUDED.Natural_Gas,
            Nuclear = EXCLUDED.Nuclear,
            Solar = EXCLUDED.Solar,
            Wind = EXCLUDED.Wind;
        '''

        con_ddb.sql(gen_cap_insert_update)

        res = con_ddb.sql('select count(*) from gen_cap')
        end_count = res.fetchall()[0][0]
        insert_count = end_count - start_count
        rows_updated = update_count - insert_count
        log.info(
            f'ROWS INSERTED: {insert_count:,} ROWS UPDATED: {rows_updated :,} TOTAL: {end_count:,}')